# 读取npy数据为某种格式

## 1.该notebook脚本说明
* 读取内容为/npy_datas中的npy文件（由上一个脚本生成，每个数据集对应生成一个）

### 1.1前置条件
如上所述

### 1.2notebook功能区域划分
* cell-1：加载所需的库以及配置
* cell-2：加载读取所需函数
* cell-3：输出测试以及验证结果可行性

### 1.3脚本执行目标
输入
```
    根文件路径，
    数据集名称，
    任务:"SIF"/"SGF"
```
输出：
```
    筛选后的X矩阵
    筛选后的Y（原始值）
    筛选后的Y（根据阈值二值化）
    特征名称列表
```

⚠ **Notice**：
* 在本文件开始之前，在前置notebook文件中已经进行了monomer的筛选，该文件读到的所有npy文件均是单体的信息
* 阈值，异常值可以在cell-1中设置



In [1]:
#---------------该cell是一些测试的东西，可以忽略掉--------------------------------- 


# 测试内容如下
# 需要做的东西包括：读取npy文件，根据SIF和SGF标签进行二值化筛选
# 参数：根文件路径，筛选数据集，任务类型（SIF/SGF）
# 中间需要的筛选逻辑：
    # 1. 读取X.npy, y_sif.npy, y_sgf.npy, ids.npy
    # 2. 根据is_monomer和任务类型，筛选出对应的样本
    # 3. 对y进行二值化处理（根据SIF和SGF进行二值化处理）
    # 4. 输出筛选后的X和y
# 输出：筛选后的X和被二值化的y

#!/usr/bin/env python3
# 除此之外，我比较喜欢写注释而不是markdown
import numpy as np
from pathlib import Path
#   script_dir = Path(__file__).parent  # 脚本目录,但是这个在notebook里面不能用
#   script_dir = Path().resolve()  # 当前工作目录
#   data_path = script_dir / "../outputs/npy_datas/sif_sgf_second_processed/X.npy"
#   X = np.load(data_path)
#   print(X.shape)

#   feature_names_path = script_dir / "../outputs/npy_datas/sif_sgf_second_processed/feature_names.npy"
#   feature_names = np.load(feature_names_path, allow_pickle=True).tolist()
#   print(len(feature_names))




# 测试了一下，Morgan分子指纹应该把分子值转化为numpy里面的布尔数值了
# 合理猜测，Aovlan也是一样的处理方式
# 注意feature_names里面的列名是字符串类型的，并且是Morgan之类的，而不是原本csv里面的列名，所以到了这一步基本就无法读取是否为单体分子了




## 2.Cell-1：导入数据以及配置相关库文件

In [2]:
# 库函数
from pathlib import Path
import numpy as np

CONFIG = {
    'npy_datas_dir': Path().resolve() / 'npy_datas',
    'dataset_names': ['sif_sgf_second', 'US9624268', 'US9809623B2','US20140294902A1', 'WO2017011820A2'],
    'dataset': {
        'sif_sgf_second': 'sif_sgf_second_processed',
        'US9624268': 'US9624268_processed',
        'US9809623B2': 'US9809623B2_processed',
        'US20140294902A1': 'US20140294902A1_processed',
        'WO2017011820A2': 'WO2017011820A2_processed',
    },
    'abnormal_point':700,
    'threshold'   : {
        'sif': 270,
        'sgf': 250,
    }

}

print("加载数据成功")
print("判断所有数据均存在...")
for dataset_name in CONFIG['dataset_names']:
    dataset_dir = CONFIG['npy_datas_dir'] / CONFIG['dataset'][dataset_name]
    required_files = ['X.npy', 'y_sif.npy', 'y_sgf.npy', 'feature_names.npy']
    for file_name in required_files:
        file_path = dataset_dir / file_name
        if not file_path.exists():
            raise FileNotFoundError(f"缺少文件: {file_path}")
    print(f"{dataset_name} 的npy数据文件存在。")
print("所有数据文件均存在。")

加载数据成功
判断所有数据均存在...
sif_sgf_second 的npy数据文件存在。
US9624268 的npy数据文件存在。
US9809623B2 的npy数据文件存在。
US20140294902A1 的npy数据文件存在。
WO2017011820A2 的npy数据文件存在。
所有数据文件均存在。


## 3.Cell-2：筛选方法生成数据和信息

最后获得结果为ndarray数据格式，其中包括X，原始y值，二值化y值，特征名称
* 1. 获得数据默认做了如下处理：去除所有非单体，去掉所有对应任务半衰期（y值）为nan数据，去掉与当前任务无关的数据
* 2. 该方法获得数据的意义为：在【某数据集】中所有可用于【某任务】的，满足【非异常值】【非单体】和【半衰期非nan】的数据

In [3]:

# 该方法的返回值为筛选后的X（特定数据集，特定的任务，以及异常值筛选），二值化后的y（根据特定任务进行阈值划分），以及特征名称列表

def load_and_filter_from_npy(
    root_dir: Path,         # 数据集的根文件路径，这里只要outputs/npy_datas就行
    dataset_name: str,      # 数据集的名称即可，_processed之类的后缀不需要,代码里面会加上的
    target: str             # "SIF" or "SGF"，大写与否都无所谓
):
    """
    从 NPY 文件中读取数据，根据任务类型和 is_monomer 进行筛选，
    并对标签进行二值化处理。

    Returns
    -------
    X_filtered : np.ndarray, shape (M, D)
    y_binary   : np.ndarray, shape (M,)
    feature_names : np.ndarray or list
    """
    # 1.读取数据
    dataset_dir = root_dir / dataset_name
    X = np.load(dataset_dir / "X.npy")
    y_sif = np.load(dataset_dir / "y_sif.npy")
    y_sgf = np.load(dataset_dir / "y_sgf.npy")
    feature_names = np.load(dataset_dir / "feature_names.npy", allow_pickle=True)

    # 一致性检查，如果不通过就报错
    n = X.shape[0]
    assert y_sif.shape[0] == n
    assert y_sgf.shape[0] == n

    # 2. 找到 is_monomer 在 X 中对应的列号-----这个后面根据序号进行扩充吧,该方法已经被完全废弃，因为筛选已经移动到csv中
    #feature_names = list(feature_names)
    #if "is_monomer" not in feature_names:
    #    raise ValueError("feature_names 中未找到 'is_monomer' 特征")
    #monomer_col_idx = feature_names.index("is_monomer")

    y_raw = None
    threshold = None
    abnormal_point = CONFIG['abnormal_point']

    # 3. 选择任务标签
    if target.upper() == "SIF":
        y_raw = y_sif
        threshold = CONFIG['threshold']['sif']
    elif target.upper() == "SGF":
        y_raw = y_sgf
        threshold = CONFIG['threshold']['sgf']
    else:
        raise ValueError("target 必须是 'SIF' 或 'SGF'")

    
    # ------------------------------------------------------------------
    # 4和5用了三个很有意思的特性， csv转npy时候的自动转化操作，numpy的筛选操作，多重叠加的按位与操作
    # ------------------------------------------------------------------

    # 4. 构造样本筛选 mask
    valid_mask = np.ones(n, dtype=bool)  # 生成的原址数值全都为1

    # 4.1 根据 is_monomer 过滤（如果指定），该方法已经被完全废弃，因为筛选已经移动到csv中
    #if is_monomer is not None:
    #    valid_mask &= (X[:, monomer_col_idx] == int(is_monomer)) # 如果是单体，则筛选出单体样本，否则筛选出非单体样本

    # 4.2 过滤无效标签(mask是按位与操作)
    valid_mask &= (y_raw != -1)   # 添加筛选条件，若为-1则过滤
    valid_mask &= ~np.isnan(y_raw) # 添加筛选条件，如果是nan则过滤掉
    valid_mask &= (y_raw <= abnormal_point)  # 添加筛选条件，若大于异常值则过滤掉

    # 5. 应用筛选（按位置删除）
    X_valid = X[valid_mask]
    y_valid = y_raw[valid_mask]

    print(f"筛选后样本数: {X_valid.shape[0]} / {X.shape[0]}")

    # 6. 二值化标签，半衰期大于阈值则视为稳定，否则视为不稳定
    y_binary = (y_valid >= threshold).astype(np.int32)

    print("============================================================================")
    print(f"根据任务{target}筛选数据集 {dataset_name} 后的统计信息:")
    print(f"  样本数: {X_valid.shape[0]}")
    print(f"  本次筛选设定阈值为: {threshold} 分钟，异常值为{abnormal_point}分钟")
    print(f"  稳定/不稳定: {(y_binary == 1).sum()} / {(y_binary == 0).sum()}")
    print(f"    符合该条件的数据条数：{X_valid.shape[0]}，特征维度: {X_valid.shape[1]}")
    print(f"    特征名称示例: {feature_names[:5]} ...")
    print(f"    {target}半衰期范围: {y_valid.min()} - {y_valid.max()}")
    print("============================================================================")

    return X_valid,y_valid, y_binary, feature_names

## 4.cell-3：代码测试以及方法使用说明

大致使用方法

In [4]:
### 方法读取示例
X_filtered,y_filtered, y_binary, feature_names = load_and_filter_from_npy(
    root_dir=CONFIG['npy_datas_dir'],   
    dataset_name=CONFIG['dataset']['US9624268'],
    target='SIF'
)


print("筛选后数据形状和类型:")
print(X_filtered.shape,y_filtered.shape, y_binary.shape, len(feature_names))
print(type(X_filtered),type(y_filtered), type(y_binary), type(feature_names))

筛选后样本数: 130 / 130
根据任务SIF筛选数据集 US9624268_processed 后的统计信息:
  样本数: 130
  本次筛选设定阈值为: 270 分钟，异常值为700分钟
  稳定/不稳定: 76 / 54
    符合该条件的数据条数：130，特征维度: 536
    特征名称示例: ['QED_MW' 'QED_ALOGP' 'QED_HBA' 'QED_HBD' 'QED_PSA'] ...
    SIF半衰期范围: 30 - 420
筛选后数据形状和类型:
(130, 536) (130,) (130,) 536
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
